In [1]:
import pandas as pd
from pycaret.classification import *
import os

In [15]:
pd.set_option('display.max_columns', None)

In [2]:
screenshots = pd.read_csv("images/screenshots/screenshots.csv")

In [5]:
screenshots_ph = pd.read_csv("images/screenshots_ph/screenshots_ph.csv")

In [6]:
regulars = pd.read_csv("images/regular/regular.csv")

In [7]:
not_good = pd.read_csv("images/not_good/not_good.csv")

In [8]:
superb = pd.read_csv("images/superb/superb.csv")

In [9]:
docs = pd.read_csv("images/docs/docs.csv")

In [10]:
receipts = pd.read_csv("images/receipts/receipts.csv")

In [11]:
test = pd.read_csv("images/test/test.csv")

In [80]:
regulars["search"] = 1
not_good["search"] = 0
superb['search'] = 0
screenshots["search"] = 0
docs["search"] = 0
receipts["search"] = 0
screenshots_ph["search"] = 0

In [81]:
whole_docs = pd.concat([regulars, not_good, superb, screenshots, docs, screenshots_ph, receipts], ignore_index=True)

In [82]:
whole_docs.describe()

,size_w,size_h,ch,aspect_ratio,mean_value_b,mean_value_g,mean_value_r,median_value_b,median_value_g,median_value_r,...,faces,has_text,filesize,sharpness,gradient,texture_energy,texture_contrast,color_balance,focus_score,search
count,886.000000,886.000000,886.0,886.000000,886.000000,886.000000,886.000000,886.000000,886.000000,886.000000,...,886.000000,8.860000e+02,8.860000e+02,886.000000,886.000000,886.000000,886.0,886.000000,886.000000,886.000000
mean,2436.980813,2333.952596,3.0,1.044080,131.391750,138.103901,141.920481,134.562077,142.846501,147.215576,...,0.878104,3.926873e+06,2.346251e+06,1029.844250,11721.555623,0.061518,0.0,55.134190,11721.555623,0.185102
std,1411.509671,1442.623656,0.0,0.473602,55.232666,50.670403,49.824859,69.839979,63.857408,64.208469,...,2.181803,3.903384e+06,3.432471e+06,2138.196428,16013.879912,0.157114,0.0,16.995918,16013.879912,0.388599
min,220.000000,360.000000,3.0,0.373750,0.121475,0.121475,0.121475,0.000000,0.000000,0.000000,...,0.000000,1.552300e+04,3.742000e+03,0.009402,0.121187,0.000040,0.0,0.098913,0.121187,0.000000
25%,1280.000000,1080.000000,3.0,0.707361,91.434778,107.801650,110.574248,81.000000,102.250000,106.000000,...,0.000000,7.623401e+05,1.973235e+05,56.246662,1297.364181,0.000664,0.0,43.749004,1297.364181,0.000000
50%,2246.000000,1839.500000,3.0,0.750000,129.689442,135.568188,143.016605,133.000000,141.000000,154.000000,...,0.000000,2.193298e+06,5.303345e+05,302.459219,5515.896704,0.001570,0.0,56.414677,5515.896704,0.000000
75%,3840.000000,3120.000000,3.0,1.336082,165.100628,168.130024,172.857088,183.000000,185.000000,188.750000,...,1.000000,6.408875e+06,3.370682e+06,1250.458838,15624.823715,0.011777,0.0,65.972867,15624.823715,0.000000
max,6720.000000,6720.000000,3.0,3.598456,253.462925,253.322823,253.105211,255.000000,255.000000,255.000000,...,43.000000,1.968548e+07,1.754783e+07,30170.659310,177580.040200,0.991532,0.0,115.693060,177580.040200,1.000000


In [83]:
whole_docs.keys()

Index(['size_w', 'size_h', 'ch', 'aspect_ratio', 'mean_value_b',
       'mean_value_g', 'mean_value_r', 'median_value_b', 'median_value_g',
       'median_value_r', 'std_deviation_b', 'std_deviation_g',
       'std_deviation_r', 'contrast', 'brightness', 'hog_features_mean',
       'hog_features_median', 'hog_features_std', 'hog_features_len', 'h_mean',
       'h_median', 'h_std', 's_mean', 's_median', 's_std', 'v_mean',
       'v_median', 'v_std', 'n_objects', 'area', 'area_mean', 'area_std',
       'vert', 'vert_mean', 'vert_std', 'color_mean', 'color_std', 'corners',
       'lines', 'faces', 'has_text', 'filesize', 'depth', 'sharpness',
       'gradient', 'texture_energy', 'texture_contrast', 'color_balance',
       'focus_score', 'filename', 'search'],
      dtype='object')

In [84]:
whole_docs = whole_docs.drop(columns=["filename", "depth", "texture_contrast", "ch"])
whole_docs.sample()

,size_w,size_h,aspect_ratio,mean_value_b,mean_value_g,mean_value_r,median_value_b,median_value_g,median_value_r,std_deviation_b,...,lines,faces,has_text,filesize,sharpness,gradient,texture_energy,color_balance,focus_score,search
663,1200,1600,1.333333,157.343115,162.122862,166.625378,168.0,173.0,178.0,36.921716,...,23048,0,1755374.5,205632,906.746138,14417.795778,0.002808,36.669085,14417.795778,0


In [85]:
corr = whole_docs.corr()


In [86]:
corr.search.sort_values()


color_mean            -0.189735
median_value_b        -0.185446
texture_energy        -0.179564
median_value_g        -0.164418
mean_value_b          -0.158330
median_value_r        -0.154169
brightness            -0.149676
mean_value_g          -0.145748
mean_value_r          -0.141465
v_median              -0.119322
hog_features_std      -0.118727
area_mean             -0.116165
area_std              -0.111824
v_mean                -0.108860
focus_score           -0.044558
gradient              -0.044558
aspect_ratio          -0.025725
has_text              -0.022736
vert_mean             -0.016825
area                  -0.015955
filesize               0.001516
hog_features_len       0.003205
sharpness              0.020142
corners                0.023864
size_w                 0.024224
size_h                 0.039725
h_median               0.059665
color_std              0.089202
h_mean                 0.091770
n_objects              0.096196
h_std                  0.103203
vert    

In [87]:
s = setup(whole_docs, target = 'search', session_id = 123)

,Description,Value
0,Session id,123
1,Target,search
2,Target type,Binary
3,Original data shape,"(886, 47)"
4,Transformed data shape,"(886, 47)"
5,Transformed train set shape,"(620, 47)"
6,Transformed test set shape,"(266, 47)"
7,Numeric features,46
8,Rows with missing values,1.2%
9,Preprocess,True


In [88]:
best_not_good = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9194,0.9425,0.6947,0.8561,0.7586,0.7112,0.7225,0.4080
catboost,CatBoost Classifier,0.9194,0.9474,0.6803,0.8765,0.7530,0.7066,0.7233,2.9650
xgboost,Extreme Gradient Boosting,0.9177,0.9352,0.6795,0.8603,0.7509,0.7029,0.7157,0.0430
et,Extra Trees Classifier,0.9145,0.9419,0.6197,0.8954,0.7236,0.6760,0.6972,0.0490
rf,Random Forest Classifier,0.9065,0.9235,0.5758,0.8982,0.6863,0.6360,0.6667,0.0720
gbc,Gradient Boosting Classifier,0.9065,0.9312,0.6439,0.8288,0.7153,0.6608,0.6747,0.1880
ada,Ada Boost Classifier,0.8871,0.9175,0.6530,0.7380,0.6798,0.6128,0.6232,0.0530
lda,Linear Discriminant Analysis,0.8500,0.8687,0.5326,0.6207,0.5622,0.4738,0.4826,0.0060
ridge,Ridge Classifier,0.8339,0.0000,0.3061,0.6171,0.3927,0.3133,0.3457,0.0060
lr,Logistic Regression,0.8258,0.8365,0.3500,0.6051,0.4159,0.3254,0.3541,0.0440


In [ ]:
tune_model(best_not_good)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:21:29
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [19]:
ids_test = test.pop("filename")

In [20]:
not_good_pred = predict_model(best_not_good, data=test)

In [21]:
not_good_pred.sample()

,size_w,size_h,ch,aspect_ratio,mean_value_b,mean_value_g,mean_value_r,median_value_b,median_value_g,median_value_r,...,filesize,depth,sharpness,gradient,texture_energy,texture_contrast,color_balance,focus_score,prediction_label,prediction_score
60,2016,4032,3,2.0,124.210114,140.406235,155.223267,121.0,137.0,155.0,...,2516117,uint8,58.429031,4398.277344,0.001756,0.0,49.799664,4398.277344,0,0.9994


In [22]:
not_good_pred["name"] = ids_test

In [23]:
not_good_photos = not_good_pred.loc[not_good_pred.prediction_label == 1 ,["name", "prediction_label", "prediction_score"]]

In [24]:
not_good_photos

,name,prediction_label,prediction_score
8,regular (10).JPG,1,0.9976
24,not_good (8).JPG,1,0.9347
33,not_good (5).JPG,1,0.9650
42,not_good (9).JPG,1,0.9986
44,superb (1).jpg,1,0.5244
49,not_good (6).JPG,1,0.9911
62,not_good (7).JPG,1,0.7597
63,regular (9).JPG,1,0.7655
64,regular (15).JPG,1,0.6236
68,regular (5).JPG,1,0.9968


In [25]:
if not os.path.isdir("./images/test/1/"):
    os.mkdir("./images/test/1/")
if not os.path.isdir("./images/test/0/"):
    os.mkdir("./images/test/0/") 


In [26]:
def movingImage(filename, location, destination):
    location = location + filename
    destination = destination + filename
    os.rename(location, destination)

In [27]:
for step in not_good_photos["name"]:
    movingImage(step, "./images/test/", "./images/test/1/")

In [28]:
not_bad_photos = not_good_pred.loc[not_good_pred.prediction_label == 0 ,["name", "prediction_label", "prediction_score"]]

In [29]:
for step in not_bad_photos["name"]:
    movingImage(step, "./images/test/", "./images/test/0/")

In [30]:
not_good_pred


,size_w,size_h,ch,aspect_ratio,mean_value_b,mean_value_g,mean_value_r,median_value_b,median_value_g,median_value_r,...,depth,sharpness,gradient,texture_energy,texture_contrast,color_balance,focus_score,prediction_label,prediction_score,name
0,1280,904,3,0.706250,246.206100,244.809433,244.117859,255.0,255.0,255.0,...,uint8,1841.777222,33596.929688,0.588539,0.0,40.820488,33596.929688,0,1.0000,screenshots (1).JPG
1,1280,591,3,0.461719,146.109192,153.940399,162.087952,173.0,173.0,173.0,...,uint8,2603.048096,45339.917969,0.102465,0.0,100.485756,45339.917969,0,1.0000,screenshots (10).JPG
2,878,1594,3,1.815490,222.977325,228.492493,228.665588,255.0,255.0,255.0,...,uint8,398.435486,18998.435547,0.286698,0.0,53.538616,18998.435547,0,1.0000,screenshots (1).png
3,1268,2102,3,1.657729,243.880280,243.444717,242.887512,255.0,255.0,255.0,...,uint8,2035.957520,33824.468750,0.377107,0.0,39.141560,33824.468750,0,1.0000,screenshots (10).png
4,1280,960,3,0.750000,143.283218,150.069199,144.446091,149.0,154.0,146.0,...,uint8,1328.645630,10828.695312,0.022690,0.0,21.210787,10828.695312,0,1.0000,docs (15).JPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1270,2362,3,1.859843,235.082870,236.869186,235.545029,255.0,255.0,255.0,...,uint8,990.406067,15578.054688,0.323792,0.0,46.956673,15578.054688,0,1.0000,screenshots (13).png
86,1038,1850,3,1.782274,247.124176,247.132874,247.097626,255.0,255.0,255.0,...,uint8,1777.006836,32144.330078,0.894077,0.0,41.519806,32144.330078,0,1.0000,screenshots (2).png
87,1600,1200,3,0.750000,139.222885,150.281479,157.251343,165.0,164.0,164.0,...,uint8,176.958405,5317.399414,0.006446,0.0,39.940491,5317.399414,0,1.0000,docs (16).JPG
88,4032,3024,3,0.750000,66.038475,62.667686,69.149940,57.0,56.0,62.0,...,uint8,77.186325,543.351868,0.001137,0.0,51.208015,543.351868,1,0.9966,not_good (1).JPG


In [31]:
evaluate_model(best_not_good)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [32]:
not_bad_photos.loc[not_bad_photos.name == "2024-01-18 14-43-46.JPG"]

,name,prediction_label,prediction_score


In [56]:
not_good_pred.loc[not_good_pred.name == "2019-07-16 14-21-46.JPG"]

,size_w,size_h,ch,aspect_ratio,mean_value_b,mean_value_g,mean_value_r,median_value_b,median_value_g,median_value_r,...,depth,sharpness,gradient,texture_energy,texture_contrast,color_balance,focus_score,prediction_label,prediction_score,name
